### Install and import libaries
References (detailed links to functions and examples in each cell):
- Semantic Link Labs library: https://github.com/microsoft/semantic-link-labs
    - Wiki: https://github.com/microsoft/semantic-link-labs/wiki
    - sempy_labs package: https://semantic-link-labs.readthedocs.io/en/latest/sempy_labs.html
    - Code Examples: https://github.com/microsoft/semantic-link-labs/wiki/Code-Examples
    - Helper Notebooks: https://github.com/microsoft/semantic-link-labs/tree/main/notebooks
#
- Semantic Link: https://learn.microsoft.com/en-us/fabric/data-science/semantic-link-overview
    - Sempy Package: https://learn.microsoft.com/en-us/python/api/semantic-link-sempy/sempy?view=semantic-link-python
    - Fabric Functions Code Examples: https://github.com/m-kovalsky/Fabric

This notebook utilizes the Semantic Link Labs library to dynamically deploy Power BI semantic models and reports across various workspaces. This specific use case involves maintaining a single master model and report, which are then duplicated across multiple workspaces and Lakehouses to accommodate diverse data sources (e.g., by store). Key benefits include centralized governance, automated deployments, separate source tables (potentially smaller row counts), and reduced need for Row-Level Security (RLS), thereby enhancing performance.


In [ ]:
# Install the library in a Fabric notebook: https://github.com/microsoft/semantic-link-labs?tab=readme-ov-file#install-the-library-in-a-fabric-notebook

%pip install semantic-link-labs

In [2]:
# Import the libraries: https://github.com/microsoft/semantic-link-labs?tab=readme-ov-file#once-installed-run-this-code-to-import-the-library-into-your-notebook

import sempy.fabric as fabric
import sempy_labs as labs
from sempy_labs import migration, directlake, admin, graph
from sempy_labs import lakehouse as lake
from sempy_labs import report as rep
from sempy_labs.tom import connect_semantic_model
from sempy_labs.report import ReportWrapper

StatementMeta(, afe51d3b-fea3-460a-b616-3a848e72eebd, 10, Finished, Available, Finished)

### Programmatically deploy semantic models and reports with Lakehouse schemas

1. Deploy semantic model to new workspace and rename
2. Update semantic model connection to new Lakehouse
3. (Optional) Check semantic model Lakehouse connection
4. Update Direct Lake table partition to new schema
5. (Optional) Get Tabular Model Scripting Language (TMSL) to confirm lineage
6. Clone report to new workspace and rebind to new semantic model
7. (Optional) Launch report to preview

In [3]:
# Define parameters

stores = ["Asia", "NorthAmerica"]
source_dataset = 'Contoso Retail - Europe'
source_dataset_workspace = 'Data Hub - Europe'
source_report = 'Contoso Sales Report - Europe'
source_report_workspace = 'Reporting Hub - Europe'

StatementMeta(, afe51d3b-fea3-460a-b616-3a848e72eebd, 11, Finished, Available, Finished)

In [4]:
# Initialize lists for each parameter

target_datasets = []
target_workspaces = []
target_lakehouses = []
target_lakehouse_workspaces = []
target_schemas = []
target_reports = []
target_report_workspaces = []

# Populate lists

for store in stores:
    target_datasets.append(f"Contoso Retail - {store}")
    target_workspaces.append(f"Data Hub - {store}")
    target_lakehouses.append("ReportingLakehouse")
    target_lakehouse_workspaces.append(f"Data Hub - {store}")
    target_schemas.append(store)
    target_reports.append(f"Contoso Sales Report - {store}")
    target_report_workspaces.append(f"Reporting Hub - {store}")

StatementMeta(, afe51d3b-fea3-460a-b616-3a848e72eebd, 12, Finished, Available, Finished)

In [5]:
# Deploy semantic model to new workspace

"""
    Function - deploy_semantic_model: https://semantic-link-labs.readthedocs.io/en/stable/sempy_labs.html#sempy_labs.deploy_semantic_model 
        Code Example: https://semantic-link-labs.readthedocs.io/en/stable/sempy_labs.html#sempy_labs.deploy_semantic_model
"""

import sempy_labs as labs

refresh_target_dataset = False
overwrite = True

for i in range(len(stores)):
    target_dataset = target_datasets[i]
    target_workspace = target_workspaces[i]
    
    labs.deploy_semantic_model(
        source_dataset = source_dataset,
        source_workspace = source_dataset_workspace,
        target_dataset = target_dataset,
        target_workspace = target_workspace,
        refresh_target_dataset = refresh_target_dataset,
        overwrite = overwrite
    )

StatementMeta(, afe51d3b-fea3-460a-b616-3a848e72eebd, 13, Finished, Available, Finished)

🟢 The 'Contoso Retail - Asia' semantic model has been updated within the 'Data Hub - Asia' workspace.
🟢 The 'Contoso Retail - NorthAmerica' semantic model has been updated within the 'Data Hub - NorthAmerica' workspace.


In [6]:
# Update semantic model connection to new Lakehouse

"""
     Function - update_direct_lake_model_lakehouse_connection: https://semantic-link-labs.readthedocs.io/en/stable/sempy_labs.directlake.html#sempy_labs.directlake.update_direct_lake_model_lakehouse_connection
"""

from sempy_labs.directlake import update_direct_lake_model_lakehouse_connection

for i in range(len(stores)):
    target_dataset = target_datasets[i]
    target_workspace = target_workspaces[i]
    target_lakehouse = target_lakehouses[i]
    target_lakehouse_workspace = target_lakehouse_workspaces[i]

    update_direct_lake_model_lakehouse_connection(
         dataset = target_dataset,
         workspace =  target_workspace,
         lakehouse =  target_lakehouse,
         lakehouse_workspace = target_lakehouse_workspace
    )

StatementMeta(, afe51d3b-fea3-460a-b616-3a848e72eebd, 14, Finished, Available, Finished)

🟢 The expression in the 'Contoso Retail - Asia' semantic model has been updated to point to the 'ReportingLakehouse' lakehouse in the 'Data Hub - Asia' workspace.
🟢 The expression in the 'Contoso Retail - NorthAmerica' semantic model has been updated to point to the 'ReportingLakehouse' lakehouse in the 'Data Hub - NorthAmerica' workspace.


In [7]:
# (Optional) Check Lakehouse connection

"""
   Function - get_direct_lake_lakehouse: https://semantic-link-labs.readthedocs.io/en/stable/sempy_labs.directlake.html#sempy_labs.directlake.get_direct_lake_lakehouse
"""

import sempy_labs as labs

for i in range(len(stores)):
   target_dataset = target_datasets[i]
   target_workspace = target_workspaces[i]
   x = labs.directlake.get_direct_lake_lakehouse(dataset=target_dataset, workspace=target_workspace)
   print(x)

StatementMeta(, afe51d3b-fea3-460a-b616-3a848e72eebd, 15, Finished, Available, Finished)

('ReportingLakehouse', '1a723c04-7284-491d-8382-27c99b5c3bee')
('ReportingLakehouse', 'f76f6f55-c183-4575-9c4b-cbfd617af9e6')


In [8]:
# Loop tables in model to update to new Lakehouse schema

"""
    Function - list_tables: https://learn.microsoft.com/en-us/python/api/semantic-link-sempy/sempy.fabric?view=semantic-link-python
        Code Example: https://github.com/m-kovalsky/Fabric#list-the-tables-within-a-given-dataset-semantic-model
    Function - update_direct_lake_partition_entity: https://semantic-link-labs.readthedocs.io/en/stable/sempy_labs.directlake.html#sempy_labs.directlake.update_direct_lake_partition_entity
        Code Example: https://github.com/microsoft/semantic-link-labs/wiki/Code-Examples#update-a-tablepartition-to-point-to-a-different-lakehouse-table
"""

import sempy
import sempy.fabric as fabric
import sempy_labs as labs
import pandas as pd

for i in range(len(stores)):
    target_dataset = target_datasets[i]
    target_workspace = target_workspaces[i]
    target_lakehouse = target_lakehouses[i]
    target_schema = target_schemas[i]
    
    # Get the list of tables in the model
    tables = fabric.list_tables(dataset = target_dataset, workspace = target_workspace)
    
    # Convert the list of tables to a DataFrame
    tables_df = pd.DataFrame(tables, columns=['Name'])
    
    # Loop through each table and update the partition entity
    for table_name in tables_df['Name']:
        entity_name = table_name
        labs.directlake.update_direct_lake_partition_entity(
            dataset = target_dataset,
            table_name = table_name,
            entity_name = entity_name,
            schema = target_schema,
            workspace = target_workspace
        )

StatementMeta(, afe51d3b-fea3-460a-b616-3a848e72eebd, 16, Finished, Available, Finished)

🟢 The 'DimStore' table in the 'Contoso Retail - Asia' semantic model within the 'Data Hub - Asia' workspace has been updated to point to the 'DimStore' table.
🟢 The 'DimCustomer' table in the 'Contoso Retail - Asia' semantic model within the 'Data Hub - Asia' workspace has been updated to point to the 'DimCustomer' table.
🟢 The 'DimDate' table in the 'Contoso Retail - Asia' semantic model within the 'Data Hub - Asia' workspace has been updated to point to the 'DimDate' table.
🟢 The 'RLS_Product' table in the 'Contoso Retail - Asia' semantic model within the 'Data Hub - Asia' workspace has been updated to point to the 'RLS_Product' table.
🟢 The 'DimProduct' table in the 'Contoso Retail - Asia' semantic model within the 'Data Hub - Asia' workspace has been updated to point to the 'DimProduct' table.
🟢 The 'FactOnlineSalesAggregated' table in the 'Contoso Retail - Asia' semantic model within the 'Data Hub - Asia' workspace has been updated to point to the 'FactOnlineSalesAggregated' table

In [9]:
# Reframe Semantic Model - only needed if automatic reframing is disabled

"""
    Function - refresh_semantic_model: https://semantic-link-labs.readthedocs.io/en/stable/sempy_labs.html#sempy_labs.refresh_semantic_model
        Code Example: https://github.com/microsoft/semantic-link-labs/wiki/Code-Examples#refresh-a-semantic-model
        Code Example Detailed: https://github.com/microsoft/semantic-link-labs/blob/main/notebooks/Semantic%20Model%20Refresh.ipynb
"""

import sempy_labs as labs

for i in range(len(stores)):
    target_dataset = target_datasets[i]
    target_workspace = target_workspaces[i]
    
    labs.refresh_semantic_model(
        dataset = target_dataset,
        workspace = target_workspace
    )

StatementMeta(, afe51d3b-fea3-460a-b616-3a848e72eebd, 17, Finished, Available, Finished)

⌛ Refresh of the 'Contoso Retail - Asia' semantic model within the 'Data Hub - Asia' workspace is in progress...
🟢 Refresh 'full' of the 'Contoso Retail - Asia' semantic model within the 'Data Hub - Asia' workspace is complete.
⌛ Refresh of the 'Contoso Retail - NorthAmerica' semantic model within the 'Data Hub - NorthAmerica' workspace is in progress...
🟢 Refresh 'full' of the 'Contoso Retail - NorthAmerica' semantic model within the 'Data Hub - NorthAmerica' workspace is complete.


In [ ]:
# (Optional) Get Tabular Model Scripting Language (TMSL) to confirm lineage

""" 
    Function - get_tmsl: https://learn.microsoft.com/en-us/python/api/semantic-link-sempy/sempy.fabric?view=semantic-link-python
        Code Example: https://github.com/m-kovalsky/Fabric?tab=readme-ov-file#shows-the-tmsl-for-a-given-dataset 
"""

import sempy
import sempy.fabric as fabric

for i in range(len(stores)):
    target_dataset = target_datasets[i]
    target_workspace = target_workspaces[i]
    
    x = fabric.get_tmsl(
        dataset=target_dataset, 
        workspace=target_workspace
    )
    print(x)

In [11]:
# Clone report and bind to semantic model or update report

""" 
    Function - fabric.list_reports: https://learn.microsoft.com/en-us/python/api/semantic-link-sempy/sempy.fabric?view=semantic-link-python
        Code Example: https://github.com/m-kovalsky/Fabric#shows-a-list-of-reports-within-the-workspace
    Function - rep.clone_report: https://semantic-link-labs.readthedocs.io/en/stable/sempy_labs.report.html#sempy_labs.report.clone_report
        Code Example: https://github.com/microsoft/semantic-link-labs/wiki/Code-Examples#move-report-to-a-different-location
    Function - rep.get_report_json: https://semantic-link-labs.readthedocs.io/en/latest/sempy_labs.report.html#sempy_labs.report.get_report_json
    Function - rep.update_report_from_reportjson: https://semantic-link-labs.readthedocs.io/en/latest/sempy_labs.report.html#sempy_labs.report.update_report_from_reportjson
        Code Example: https://github.com/microsoft/semantic-link-labs/wiki/Code-Examples#move-report-to-a-different-location
"""

import sempy_labs.report as rep
import sempy.fabric as fabric
from sempy_labs.report import clone_report 

for i in range(len(stores)):
    target_report = target_reports[i]
    target_report_workspace = target_report_workspaces[i]
    target_dataset = target_datasets[i]
    target_workspace = target_workspaces[i]
    
    # Check if report exists
    dfR = fabric.list_reports(workspace=target_report_workspace)
    dfR_filt = dfR[dfR['Name'] == target_report]
    
    if dfR_filt.empty:
        # Create the report if it doesn't exist
        clone_report(
            report = source_report,
            cloned_report = target_report,
            workspace = source_report_workspace,
            target_workspace = target_report_workspace,
            target_dataset = target_dataset,
            target_dataset_workspace = target_workspace
        )
    else:
        # Update the report definition if the report already exists
        report_json = rep.get_report_json(report = source_report, workspace = source_report_workspace)
        rep.update_report_from_reportjson(
            report = target_report,
            report_json = report_json,
            workspace = target_report_workspace
        )

StatementMeta(, afe51d3b-fea3-460a-b616-3a848e72eebd, 19, Finished, Available, Finished)

🟢 The 'Contoso Sales Report - Asia' report within the 'Reporting Hub - Asia' workspace has been successfully updated.
🟢 The 'Contoso Sales Report - NorthAmerica' report within the 'Reporting Hub - NorthAmerica' workspace has been successfully updated.


In [13]:
# (Optional) Launch Asia report to preview

"""
    Function - rep.launch_report: https://semantic-link-labs.readthedocs.io/en/latest/sempy_labs.report.html#sempy_labs.report.launch_report 
"""

import sempy_labs as labs
from sempy_labs.report import launch_report

report = 'Contoso Sales Report - Asia'
workspace = 'Reporting Hub - Asia'

launch_report(report, workspace)

StatementMeta(, afe51d3b-fea3-460a-b616-3a848e72eebd, 21, Finished, Available, Finished)

Report()

StatementMeta(, afe51d3b-fea3-460a-b616-3a848e72eebd, 23, Finished, Available, Finished)

In [14]:
# (Optional) Launch North America report to preview

"""
    Function - rep.launch_report: https://semantic-link-labs.readthedocs.io/en/latest/sempy_labs.report.html#sempy_labs.report.launch_report 
"""

import sempy_labs as labs
from sempy_labs.report import launch_report

report = 'Contoso Sales Report - NorthAmerica'
workspace = 'Reporting Hub - NorthAmerica'

launch_report(report, workspace)

StatementMeta(, afe51d3b-fea3-460a-b616-3a848e72eebd, 22, Finished, Available, Finished)

Report()

StatementMeta(, afe51d3b-fea3-460a-b616-3a848e72eebd, 24, Finished, Available, Finished)

StatementMeta(, afe51d3b-fea3-460a-b616-3a848e72eebd, 25, Finished, Available, Finished)

StatementMeta(, afe51d3b-fea3-460a-b616-3a848e72eebd, 26, Finished, Available, Finished)

StatementMeta(, afe51d3b-fea3-460a-b616-3a848e72eebd, 27, Finished, Available, Finished)

Note: 
- After initial model deployment: need to update security role members, cloud connection from SSO (default) to fixed identity, access permissions.
- Once deployed and model is overwritten only cloud connection needs to be updated.